Some useful neumeric methods in python

In [27]:
import math
print(abs(-35))
print(math.ceil(2.17))
print(math.floor(8.9))
print(pow(3, 2))
print(3 ** 2) # Also acts as power operator
print(math.sqrt(25))

35
3
8
9
9
5.0


In [28]:
# Int to string
print(type(str(42)))
print(type(int("42")))

# Float conversion
print(type(str(12.3)))
print(type(float("4.88")))

<class 'str'>
<class 'int'>
<class 'str'>
<class 'float'>


Floats are not infinite percision. That's why it's better to refer "infinity" as float('inf') and float('-inf'). These values are comparable to int and can be use to create psuedo max-int and psuedo min-int value

In [29]:
num = 100
maxValue = float("inf")
if(maxValue > num):
    print("Float inf working")

Float inf working


In [30]:
#Import math Library
import math

#compare the closeness of two values
print(math.isclose(1.233, 1.4566))
print(math.isclose(1.233, 1.233))
print(math.isclose(1.233, 1.24))
print(math.isclose(1.233, 1.233000001))

False
True
False
True


## Parity
The parity of a binary number is 1 if the number of 1s is odd. If the number of 1s is even then the parity is 0

For example 1011 has parity of 1 and 110011 has parity of 0

In [35]:
# Brute force algorightm to check parity
def parity(num):
    result = 0
    while num:
        result = result ^ num & 1 # So if result is 0 and current bit is 1 then the result will be 1 due to XOR. 
        # If result is 1 and current bit is 1 then the result will be 0 due to XOR
        # Can be written as = result ^= num & 1 
        num >>= 1
    return result

print(parity(13)) #1101 => 1 * 2^3 + 1 * 2^2 + 0 * 2^1 + 1 * 2^0 => 8 + 4 + 0 + 1 => 13

1


A more efficient approach: x & (x - 1) is equals to x with it's lowest set bit erased. 
For example 6 & 5 -> 110 & 101 -> 100

So the program basically sets result to 0 and flips the value each time by XOR by overwriting it.
And drops the lowest set bit until the whole number becomes zero

In [3]:
def parityWithSetBit(x):
    result = 0
    while x: # Until x turns to zero
        result = result ^ 1 # Flips the bit
        x = x & (x - 1) # Drops the last set bit
    return result
print(parityWithSetBit(13))

1


### 4.1 Parity with Cache
For a 64 bit number use a 16 bit cache. Shift each 16 bits to 48, 32 and 16 bits right and bitmask them with a 16 bit mask. Then XOR them to find the parity

In [ ]:
def parityWithCache(x):
    maskSize = 16
    bitMask = 0xFFFF
    parity = precomputedParity[x >> (3 * maskSize)] ^ precomputedParity[(x >> (2 * maskSize)) & bitMask] ^ precomputedParity[(x >> maskSize) & bitMask] ^ precomputedParity[x & bitMask]

    return parity

Complexity is size of the keys used to index the lookup table. 
L -> Width of the words for cache -> 2.
n -> length of the word that we check parity -> 8.
Complexity is **O(n/L) -> becuase there are n/L or 4 terms to process** assuming that the word level operations takes O(1) time and without the initialisation cost of the lookup table

### Parity taking advantage of the XOR and CPU level instructions

In [15]:
# O(logN) time, where N is the word size. Because each iteration we are reducing the calculation in half
def parity(x): # 64 bit int
    # shiftBits = 0xFFFFFFFF
    # while shiftBits:
    #     x ^= x >> shiftBits
    #     shiftBits = shiftBits / 0x10
    x ^= x >> 32 # Right shift x by 32 bits and XOR that with x
    x ^= x >> 16 # x = x ^ x >> 16 Right shift x by 16 bits and XOR that with x itself
    x ^= x >> 8
    x ^= x >> 4
    x ^= x >> 2
    x ^= x >> 1
    return x & 0x1 # We only care about the last bit and that's why we are masking all the previosu bits using a bit mask & operation

print(parity(13))

1


Right propagate the rightmost set bits in x. Turn 01010000 into 01011111.

An intresting fact is that if you subtract 1 from a number like 10 or 10000 it becomes 01 or 01111. For example 16 is 10000. So 15 is basically 01111. We can use this property and make the bitwise OR operation to right propagate the numbers

In [27]:
def rightPropagate(x):
    return x | (x - 1)
print(bin(rightPropagate(16)))

0b11111


Compute x mod power of two. For example 77 mod 64 will return 13

In [29]:
def binaryModFromPowerOfTwo(x, power):
    y = pow(2, power)
    return x & (y - 1)

print(binaryModFromPowerOfTwo(13, 3))

5


In [31]:
print(bin(1 << 4))

0b10000


In [41]:
def swapBits(x, i, j):
    if (x >> i) & 1 != (x >> j) & 1: # Because we only care about the last digit. That's why we are masking all other digits by doing AND with 1. Also if both are 0 then AND with 1 will be 0 . But if both are 1 then AND with 1 will result in 1
        bitMask = (1 << i) | (1 << j) # Creating a bitmask like this 100010 to use the XOR property. 0 ^ 1 = 1 but 1 ^ 1 = 0
        x ^= bitMask
    return x

print(bin(swapBits(16, 0, 4))) # 00001
print(bin(4))

0b1
0b100


### Reverse Bits
1. Right shift
2. Bitmask unnecessary bits
3. Look up the reversed number in the look up table
4. Left shift it on it's new reversed position

In [45]:
# 11010100
# O(n/L) time - because we have to loop precomputed reverse for each bit segements
# O(L) space - where L is the cache size. Ideally n/4 
def reverseBits(x):
    # precomputedReverse -> Need to define a hashmap or set
    maskSize = 16
    bitMask = 0xFFFF
    return precomputedReverse[(x >> 3 * maskSize) & bitMask] | precomputedReverse[(x >> 2 * maskSize) & bitMask] << maskSize | precomputedReverse[(x >> maskSize) & bitMask] << 2 * maskSize | precomputedReverse[x & bitMask] << 3 * maskSize

### Find closest integer with same weight

In [52]:
# O(n) time where n is the length of the int | O(1) space
def closestIntSameBitCount(x):
    maxNumberOfBits = 64
    for i in range(maxNumberOfBits - 1): # A 64 bits int has 63 positions starting from 0. So at 62th position we will be accessing 62th and 63th position. Python range function runs until that number excluded. So in this case until 62
        if (x >> i) & 1 != (x >> (i + 1)) & 1:
            x ^= (1 << i) | (1 << (i + 1))
            return x # You have to return from the loop or break out otherwise it will keep calculating and finding the next set of unmatched bits
print(closestIntSameBitCount(6))

5
